## Import OSM Data into Python

In [46]:
import requests
import json
import re
import os
import geopandas as gpd
import fiona
import numpy as np
from typing import Optional
from shapely.ops import cascaded_union
from ipyleaflet import Map, GeoJSON, GeoData, basemaps, basemap_to_tiles, Icon, Marker, LayersControl, LayerGroup, DrawControl, FullScreenControl, MeasureControl, ScaleControl, LocalTileLayer

In [2]:
data_path = os.path.join(os.getcwd(), "..", "data/country.json")

In [3]:
with open(data_path) as json_file:
    my_dict = json.load(json_file)

In [4]:
# m = Map(basemap = basemaps.CartoDB.PositronNoLabels, center=[1.356904, 103.811978], zoom = 11, scroll_wheel_zoom=False)
# m.layout.height = "400px"
# m.layout.width = "600px"
# m.attribution_control = False

# geo_data = GeoData(geo_dataframe = singapore_gdf,
#                    style={'color': 'black', 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
#                    hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
#                    name = 'Countries')

# m.add_layer(geo_data)
# m.add_control(LayersControl())

# m

In [5]:
my_dict['Singapore']['coords']

[1.357107, 103.8194992]

In [6]:
m = Map(basemap = basemaps.CartoDB.PositronNoLabels, center=my_dict['Singapore']['coords'], zoom=11, scroll_wheel_zoom=False)
m.layout.height = "400px"
m.layout.width = "600px"
m.attribution_control = False
m

Map(center=[1.357107, 103.8194992], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Add Map Controls

In [7]:
m.add_control(FullScreenControl())
m.add_control(LayersControl(position='topright'))
            
# Configure draw control
m.add_control(DrawControl(rectangle={'shapeOptions':{'color':"#a52a2a"}},
                          polyline = {"shapeOptions": {"color": "#6bc2e5", "weight": 2, "opacity": 1.0}},
                          polygon = {"shapeOptions": {"fillColor": "#eba134", "color": "#000000", "fillOpacity": 0.5, "weight":2},
                                     "drawError": {"color": "#dd253b", "message": "Delete and redraw"},
                                     "allowIntersection": False},
                         )
             )

m.add_control(ScaleControl(position='bottomleft'))

## Add Basemaps

In [8]:
m.add_layer(basemap_to_tiles(basemaps.CartoDB.VoyagerNoLabels))
m.add_layer(LocalTileLayer(path="http://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}", name = 'Google Streets'))
m.add_layer(LocalTileLayer(path="http://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}", name = 'Google Hybrid'))
m.add_layer(LocalTileLayer(path="http://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}", name = 'Google Terrain'))

In [9]:
m

Map(center=[1.357107, 103.8194992], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title'…

## Find Bounding Box

In [21]:
def draw_bbox(m: Map):
    print('Please draw a bounding box on the map')
    display(m)

In [22]:
draw_bbox(m)

Please draw a bounding box on the map


Map(bottom=260367.0, center=[1.357107, 103.8194992], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [42]:
m.controls[3].data

[]

In [12]:
def get_bbox(m: Map):
    t_index = None
    for i, control in enumerate(m.controls):
        if isinstance(control, DrawControl):
            t_index = i
    if t_index is None:
        raise Exception('No draw control object found for map.')
    
    lon_min, lat_min, lon_max, lat_max = float('inf'), float('inf'), float('-inf'), float('-inf')
    try:
        for lon,lat in [*m.controls[3].data[0]['geometry']['coordinates'][0]]:
            lon_min = min(lon_min, lon)
            lat_min = min(lat_min, lat)
            lon_max = max(lon_max, lon)
            lat_max = max(lat_max, lat)
        m.controls[t_index].clear()
        return ([lat_min, lon_min, lat_max, lon_max])
        
    except (IndexError, TypeError):
        print('Please draw a bounding box with the rectangle, polygon, or polyline tool on the map.')
    

In [13]:
bbox = get_bbox(m)

Please draw a bounding box with the rectangle, polygon, or polyline tool on the map.


In [14]:
bbox

## Add POI Data into Layer

In [15]:
with open('country.json', 'r') as f:
  countries = json.load(f)

countries['Singapore']['A2']

FileNotFoundError: [Errno 2] No such file or directory: 'country.json'

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

def extract_osm_with_bb(bbox: Optional[list] = None, country_name: Optional[str] = None):
    
    if bbox is None:
        overpass_query = f"""
        [out:json];
        area["ISO3166-1"={countries[country_name]['A2']}][admin_level=2];
        node["amenity"="clinic"](area);
        out center;
        """
    
    else:
        overpass_query = '[out:json]; node["amenity"="clinic"]' + str(tuple(bbox))+ '; out center;'
        
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    
    data = response.json()
    return data['elements']

poi_data = extract_osm_with_bb(bbox = bbox, country_name = 'Singapore')

## Import with Geojson 

In [10]:
def project_gdf(gdf):
    mean_longitude = gdf["geometry"].representative_point().x.mean()

    # Compute UTM crs
    utm_zone = int(np.floor((mean_longitude + 180) / 6) + 1)
    utm_crs = f"+proj=utm +zone={utm_zone} +ellps=WGS84 +datum=WGS84 +units=m +no_defs"

    # project the GeoDataFrame to the UTM CRS
    gdf_proj = gdf.to_crs(utm_crs)
    print(f"Projected to {gdf_proj.crs}")
    
    return gdf_proj

In [46]:
singapore_gdf = gpd.read_file("./sg_map").to_crs('epsg:4326')

In [47]:
london_gdf = gpd.read_file("../data/london_map").to_crs('epsg:4326')

In [30]:
# proj_london = project_gdf(london_gdf)
# london_boundary = proj_london.geometry.buffer(1).unary_union
# london = gpd.GeoDataFrame(index=[0], crs = proj_london.crs, geometry=[london_boundary])
# london = london.to_crs(4236)

Projected to +proj=utm +zone=30 +ellps=WGS84 +datum=WGS84 +units=m +no_defs +type=crs


In [64]:
london = london_gdf[london_gdf['DISTRICT']=="City and County of the City of London"]

In [65]:
london

,NAME,GSS_CODE,DISTRICT,LAGSSCODE,HECTARES,NONLD_AREA,geometry
632,Castle Baynard,E05009297,City and County of the City of London,E09000001,314.943,24.546,"POLYGON ((-0.10847 51.50989, -0.10916 51.50987..."


In [66]:
london_json = london.to_json()

In [67]:
with open('./data/london.geojson', 'w') as outfile:
    outfile.write(london_json)

In [47]:
tp_zone = singapore_gdf[(singapore_gdf['LU_DESC'] == "RESIDENTIAL") & (singapore_gdf["SHAPE_Area"] == 5682579.85451)]

In [52]:
type(tp_zone)

geopandas.geodataframe.GeoDataFrame

In [ ]:
gpd.GeoDataFrame

In [59]:
type(tp_zone.geometry.values[0])

shapely.geometry.polygon.Polygon

In [ ]:
with open('geojson_string.json', 'w') as outfile:
    outfile.write(geojson_string)

In [60]:
with open('geojson_string.json') as json_file:
    my_dict = json.load(json_file)

In [62]:
with open() as json_file:
    my_dict = json.load(json_file)

In [86]:
gpd.read_file('./sg_map/G_MP14_LAND_USE_PL.shp')

,OBJECTID,OID_1,LU_DESC,LU_TEXT,GPR,WHI_Q_MX,GPR_B_MN,INC_CRC,FMEL_UPD_D,X_ADDR,Y_ADDR,SHAPE_Leng,SHAPE_Area,geometry
0,1,0,UTILITY,U,EVA,0.0,0.0,BF0C37311F485B64,2014-06-23,23226.8650,37189.3295,1474.502127,70314.109927,"POLYGON ((23311.893 37295.503, 23321.050 37278..."
1,2,0,OPEN SPACE,None,EVA,0.0,0.0,DD6E9ED08FF9F421,2014-06-23,24627.6556,36234.1907,813.738237,37851.220985,"POLYGON ((24653.610 36352.795, 24724.302 36291..."
2,3,0,OPEN SPACE,None,EVA,0.0,0.0,9A88CBC8A00238DF,2014-06-23,28064.7767,35926.0034,771.441679,12284.431868,"POLYGON ((28204.673 35972.492, 28204.150 35972..."
3,4,0,OPEN SPACE,None,EVA,0.0,0.0,2481817E30624F97,2014-06-23,26176.9607,35947.5340,1686.764546,151032.767607,"POLYGON ((26136.730 35673.236, 26138.818 35678..."
4,5,0,ROAD,None,None,0.0,0.0,C56C3E0EF8A3E0CF,2014-06-23,28179.4427,35988.4606,192.493764,1162.365423,"POLYGON ((28224.064 35984.721, 28204.673 35972..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111520,111521,0,RESIDENTIAL,None,1.4,0.0,0.0,7B395DFEDEDA6027,2014-06-23,36353.2709,33149.3208,111.846016,491.765336,"POLYGON ((36366.563 33129.680, 36356.408 33127..."
111521,111522,0,RESIDENTIAL,None,1.4,0.0,0.0,FAD02777AEE960DC,2014-06-23,36479.7057,33368.9370,76.295655,279.274408,"POLYGON ((36495.235 33366.739, 36495.176 33366..."
111522,111523,0,RESIDENTIAL,None,1.4,0.0,0.0,D3D09E2FF834B910,2014-06-23,36490.5899,33342.9947,71.612146,210.860206,"POLYGON ((36505.941 33342.008, 36505.889 33341..."
111523,111524,0,RESIDENTIAL,None,1.4,0.0,0.0,DE13A2A035ABF203,2014-06-23,36466.5639,33277.5047,94.795051,341.742823,"POLYGON ((36478.573 33260.881, 36469.639 33258..."


In [67]:
from shapely.geometry import shape
geom = [my_dict['features'][0]['geometry']['type'] for feature in my_dict]
out = gpd.GeoDataFrame({'geometry':geom})

AttributeError: 'str' object has no attribute 'get'

In [ ]:
def get_bbox(geojson: dict):
    
    lon_min, lat_min, lon_max, lat_max = float('inf'), float('inf'), float('-inf'), float('-inf')
    for lon,lat in [*geojson['features'][0]['geometry']['coordinates'][0]]:
        lon_min = min(lon_min, lon)
        lat_min = min(lat_min, lat)
        lon_max = max(lon_max, lon)
        lat_max = max(lat_max, lat)
    return ([lat_min, lon_min, lat_max, lon_max])

bbox = get_bbox(my_dict)

In [ ]:
overpass_url = "http://overpass-api.de/api/interpreter"

def extract_osm_with_geojson(geojson: Optional[dict] = None, country_name: Optional[str] = None):
    
    if geojson is None:
        overpass_query = f"""
        [out:json];
        area["ISO3166-1"={countries[country_name]['A2']}][admin_level=2];
        node["amenity"~"clinic|restaurant"](area);
        out center;
        """
    # find bounds, filter by geodataframe, return
    else:
        
        bbox = get_bbox(geojson)
        print(bbox)
        overpass_query = '[out:json]; node["amenity"~"clinic|restaurant"]' + str(tuple(bbox))+ '; out center;'
        
    response = requests.get(overpass_url, 
                            params={'data': overpass_query})
    
    data = response.json()
    return data['elements']

poi_data = extract_osm_with_geojson(geojson = my_dict, country_name = 'Singapore')

In [ ]:
type(poi_data)

In [ ]:
def convert_json(poi_data: dict):
    return ({ "type": "FeatureCollection",
                        "features": [ 
                                        {"type": "Feature",
                                         "geometry": { "type": "Point",
                                                       "coordinates": [ feature['lon'],
                                                                        feature['lat']]},
                                         "properties": { key: value 
                                                         for key, value in feature.items()
                                                         if key not in ('lat', 'lon') }
                                         } 
                                     for feature in poi_data
                                    ]
                       })

out = convert_json(poi_data)

In [ ]:
type(out)

In [ ]:
with open('out.geojson', 'w') as fp:
    json.dump(out, fp)

In [ ]:
geodf = gpd.read_file('./out.geojson')
geodf

In [ ]:
poi_layer = []
for feat in poi_data:
    icon = Icon(icon_url='https://cdn-icons-png.flaticon.com/512/7500/7500224.png', icon_size=[20, 20])
    marker = Marker(location=(feat['lat'] , feat['lon']), icon=icon)
    poi_layer.append(marker)
    layer_group = LayerGroup(layers=poi_layer, name = 'POI')
    
m.add_layer(layer_group)

In [ ]:
m

In [ ]:
class OSMLayer:
    '''
    Returns a layer of OSM elements based on user input
    '''
    def __init__(self, bbox: Optional(tuple), area: Optional(str)):
        self.area = f"area['ISO3166-1'={area}][admin_level=2]
        self.bbox = 
    
    def __repr__(self):
        